# load import libraries

In [1]:
#Aim is 1-D receiver function analysis
import numpy as np
import math
import obspy
from seispy.decon import RFTrace
import seispy
import scipy
import scipy.io as sio
from matplotlib import pyplot as plt
from future.utils import native_str
import os
import sys
from scipy import interpolate
from scipy import signal
from obspy import read
import pandas as pd

# Set All Parameters

# Code, Function[dseis] = decon(fseis,source,water_level)
# RF function

In [2]:
def reciever_function(trace,source,water_level=1000,tshift=10):
    '''
    reciever_function(trace,source,water_level=1000,tshift=10),input the seismic Trace(E,N,Z) direction 
    and rotate it in (R-T-Z)component,
    source and trace are the single traces, producing a single deconvolved trace. Where each trace in input 
    Trace is deconvohttps://github.com/PATH-TO/REPOSITORYlved by each corresponding Source 
    and apply ifft,(the value of water_level(damping factor) and tshift have been
    recommended).Having all to make
    Reciever_function for the station.
    '''
    slength = (trace.data).shape[0]
    source_matrix = np.matrix(source)
    nsrc = source_matrix.shape[0]
    trace_matrix = np.matrix(trace)
    ntr = trace_matrix.shape[0]
    n2=2**(math.ceil(math.log(slength)/math.log(2)))
    omega = np.arange(0,n2/2)*2*(math.pi)/(n2*delta)
    wtshift = -(1j)*omega*tshift
    traceft = np.fft.fft([st_rot.traces[1].data],n2,axis=1)
    sourceft = np.fft.fft([st_rot.traces[2].data],n2,axis=1)
    traceft = traceft[:,0:int(n2/2)]
    sourceft = sourceft[:,0:int(n2/2)]
    traceft_real = (traceft.real)[0]
    sourceft_real = (sourceft.real)[0]
    dseisft = np.multiply(traceft,np.conjugate(sourceft))/(np.multiply(sourceft,np.conjugate(sourceft))+water_level)
    dseisft_real = (dseisft.real)[0]
    zeros_wtshift = np.zeros_like(wtshift)
    dseisft=np.multiply(dseisft,(np.ones((1,1))*np.exp(zeros_wtshift)))
    dseisfttrans = np.array(np.transpose(dseisft))
    dseisftfinal =np.concatenate([dseisfttrans,np.fliplr(np.conjugate(dseisfttrans[1:int(n2/2)]))])
    rf = np.fft.irfft(dseisftfinal,n2,0)
    rf = (rf[0:slength])
    return(rf,omega,wtshift,dseisft_real) 

In [6]:
st = obspy.read('/home/zahra/reciever function/poin1/CN.POIN..HH*_2018-12-05T04_32_12.148_2018-12-05T04_43_12.148.sac')
print(st)
tr = st[0]
print(tr.stats)
delta =0.01
water_level=1000
da = seispy.distaz(st[0].stats.sac.stla, st[0].stats.sac.stlo,st[0].stats.sac.evla, st[0].stats.sac.evlo)
dis = da.delta
bazi = da.baz
ev_dep = st[0].stats.sac.evdp
print('Distance = %5.2f˚' %dis)
print('back_azimuth = %5.2f˚' %bazi)
st_rot = st.copy().rotate("NE->RT", back_azimuth=bazi)
trace =  st_rot.traces[1]
source = st_rot.traces[2]

rf,omega,wtshift,dseisft_real=reciever_function(trace,source,water_level)

In [5]:
plt.title("rf")
plt.plot(rf)
plt.xlim(0, 30)
plt.xticks(np.arange(0, 30,10))
plt.xlabel('Depth (km)')
plt.show()